In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkContext

# Initialize SparkContext
sc = SparkContext("local", "CitiesRDD")
sc.setLogLevel("ERROR")

In [ ]:
from google.colab import files
uploaded = files.upload()

# File should now be in the working directory
file_name = list(uploaded.keys())[0]
print("Uploaded file:", file_name)

Saving Cities_Details.csv to Cities_Details (1).csv
Uploaded file: Cities_Details (1).csv


In [ ]:
header = cities_rdd.first()
data_rdd = cities_rdd.filter(lambda row: row != header)
cities = data_rdd.map(lambda line: line.split(",")) \
                       .map(lambda x: (x[2], x[3], int(x[6])))
split_rdd = data_rdd.map(lambda row: row.split(","))
print("First 10 tuples:", cities.take(10))


First 10 tuples: [('Tokyo', 'Japan', 31480498), ('Shanghai', 'China', 14608512), ('Bombay', 'India', 12692717), ('Karachi', 'India', 11627378), ('Delhi', 'India', 10928270), ('New Delhi', 'India', 10928270), ('Manila', 'Philippines', 10443877), ('Moscow', 'Russia', 10381288), ('Seoul', 'South korea', 10323448), ('São Paulo', 'Brazil', 10021437)]


In [ ]:
countries = split_rdd.map(lambda cols: cols[3]).distinct().collect()
print("Unique Countries:", countries)

Unique Countries: ['Mexico', 'Congo', 'Bangladesh', 'Australia', 'Morocco', 'Angola', 'Italy', 'Ukraine', 'Dominican Republic', 'Syria', '', 'Brazil', 'USA', 'Peru', 'Canada', 'Myanmar', 'South Africa', 'Spain', 'Sudan', 'Tanzania�', 'Somalia', 'Iran', 'Japan', 'China', 'Philippines', 'Russia', 'South korea', 'Nigeria', 'Egypt', 'UK', 'Pakishtan', 'Afghanistan', 'Chile', 'Ivory Coast', 'Singapore', 'Saudi Arabia', 'Germany', 'Ethiopia', 'Kenya', 'Taiwan', 'France', 'India', 'Turkey', 'Indonesia', 'Colombia', 'Thailand', 'Vietnam', 'Senegal', 'Zimbabwe']


In [ ]:
cities_per_country = cities.map(lambda x: (x[1], 1)) \
                           .reduceByKey(lambda a, b: a + b)
print(cities_per_country.collect())


[('Mexico', 1), ('Congo', 1), ('Bangladesh', 1), ('Australia', 2), ('Morocco', 1), ('Angola', 1), ('Italy', 1), ('Ukraine', 1), ('Dominican Republic', 1), ('Syria', 1), ('', 998), ('Brazil', 5), ('USA', 4), ('Peru', 1), ('Canada', 2), ('Myanmar', 1), ('South Africa', 2), ('Spain', 1), ('Sudan', 1), ('Tanzania�', 1), ('Somalia', 1), ('Iran', 1), ('Japan', 2), ('China', 16), ('Philippines', 1), ('Russia', 2), ('South korea', 1), ('Nigeria', 3), ('Egypt', 3), ('UK', 1), ('Pakishtan', 1), ('Afghanistan', 2), ('Chile', 1), ('Ivory Coast', 1), ('Singapore', 1), ('Saudi Arabia', 2), ('Germany', 1), ('Ethiopia', 1), ('Kenya', 1), ('Taiwan', 1), ('France', 1), ('India', 16), ('Turkey', 3), ('Indonesia', 2), ('Colombia', 2), ('Thailand', 1), ('Vietnam', 1), ('Senegal', 1), ('Zimbabwe', 1)]


In [ ]:
top5_cities = cities.takeOrdered(5, key=lambda x: -x[2])
print(top5_cities)


[('Tokyo', 'Japan', 31480498), ('Shanghai', 'China', 14608512), ('Bombay', 'India', 12692717), ('Karachi', 'India', 11627378), ('Delhi', 'India', 10928270)]


In [ ]:
total_population = cities.map(lambda x: (x[1], x[2])) \
                         .reduceByKey(lambda a, b: a + b)
print(total_population.collect())


[('Mexico', 8720916), ('Congo', 7787832), ('Bangladesh', 6493177), ('Australia', 8124797), ('Morocco', 3609698), ('Angola', 2776125), ('Italy', 2643736), ('Ukraine', 2514227), ('Dominican Republic', 2202016), ('Syria', 2139878), ('', 659441241), ('Brazil', 23043166), ('USA', 17538900), ('Peru', 7646786), ('Canada', 7880700), ('Myanmar', 4477782), ('South Africa', 6553844), ('Spain', 3102644), ('Sudan', 2810328), ('Tanzania�', 2698651), ('Somalia', 2590180), ('Iran', 2307254), ('Japan', 33671789), ('China', 66311087), ('Philippines', 10443877), ('Russia', 14421039), ('South korea', 10323448), ('Nigeria', 15981147), ('Egypt', 13989604), ('UK', 7421228), ('Pakishtan', 6312576), ('Afghanistan', 8716105), ('Chile', 4837248), ('Ivory Coast', 3692570), ('Singapore', 3547809), ('Saudi Arabia', 6015018), ('Germany', 3398362), ('Ethiopia', 2757807), ('Kenya', 2750561), ('Taiwan', 2514794), ('France', 2110694), ('India', 85960451), ('Turkey', 15818608), ('Indonesia', 10915226), ('Colombia', 94954

In [ ]:
country_pop_count = cities.map(lambda x: (x[1], (x[2], 1))) \
                          .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))

avg_population = country_pop_count.mapValues(lambda x: x[0] / x[1])
print(avg_population.collect())


[('Mexico', 8720916.0), ('Congo', 7787832.0), ('Bangladesh', 6493177.0), ('Australia', 4062398.5), ('Morocco', 3609698.0), ('Angola', 2776125.0), ('Italy', 2643736.0), ('Ukraine', 2514227.0), ('Dominican Republic', 2202016.0), ('Syria', 2139878.0), ('', 660762.7665330662), ('Brazil', 4608633.2), ('USA', 4384725.0), ('Peru', 7646786.0), ('Canada', 3940350.0), ('Myanmar', 4477782.0), ('South Africa', 3276922.0), ('Spain', 3102644.0), ('Sudan', 2810328.0), ('Tanzania�', 2698651.0), ('Somalia', 2590180.0), ('Iran', 2307254.0), ('Japan', 16835894.5), ('China', 4144442.9375), ('Philippines', 10443877.0), ('Russia', 7210519.5), ('South korea', 10323448.0), ('Nigeria', 5327049.0), ('Egypt', 4663201.333333333), ('UK', 7421228.0), ('Pakishtan', 6312576.0), ('Afghanistan', 4358052.5), ('Chile', 4837248.0), ('Ivory Coast', 3692570.0), ('Singapore', 3547809.0), ('Saudi Arabia', 3007509.0), ('Germany', 3398362.0), ('Ethiopia', 2757807.0), ('Kenya', 2750561.0), ('Taiwan', 2514794.0), ('France', 21106

In [ ]:
max_avg_country = avg_population.takeOrdered(1, key=lambda x: -x[1])
print(max_avg_country)


[('Japan', 16835894.5)]
